In [34]:
import pandas as pd
import numpy as np

data_discover = pd.read_csv(r'Discover_Часовые_общий_актуальные_данные.csv', sep=';', na_values='N', decimal=',')

column_names_discover = ['bx_gsm', 'by_gsm', 'bz_gsm', 'bt', 'proton_density', 'proton_speed', 'proton_temperature']

In [35]:
data_discover.iloc[range(1256,1268), [4, 9, 10, 11, 20, 21]]

,bt,bx_gsm,by_gsm,bz_gsm,proton_density,proton_temperature
1256,5.807213,2.268852,-4.409508,2.625738,15.559672,218285.24590
1257,7.372292,-1.107917,-6.313542,2.603542,17.140000,94458.79167
1258,NaN,NaN,NaN,NaN,NaN,NaN
1259,NaN,NaN,NaN,NaN,NaN,NaN
1260,NaN,NaN,NaN,NaN,NaN,NaN
1261,NaN,NaN,NaN,NaN,NaN,NaN
1262,6.326842,3.803684,3.412105,3.584211,NaN,NaN
1263,5.303443,3.611967,-2.949180,2.031148,12.658125,210951.03130
1264,5.082787,3.703115,-1.958852,2.768197,13.029016,199954.37700
1265,4.494262,3.891475,-1.589672,1.217869,12.690656,199802.55740


In [50]:
pd.set_option('future.no_silent_downcasting', True)

def linear_interpolate_gaps(df, column_names, max_gap_length=12):
    for column_name in column_names:
        missing_indices = df[column_name].isnull()
        start_indices = []
        end_indices = []
        
        is_missing = False
        
        for i in range(len(df)):
            if missing_indices[i] and not is_missing:
                start_indices.append(i)
                is_missing = True
            elif not missing_indices[i] and is_missing:
                end_indices.append(i)
                is_missing = False
        
        for start, end in zip(start_indices, end_indices):
            if end - start < max_gap_length:
                if start != 0:
                    y1 = df.loc[start - 1, column_name]
                    y2 = df.loc[end, column_name]
                
                    x = np.arange(start, end)
                    y = np.interp(x, [start - 1, end], [y1, y2])
                
                    df.loc[start:end-1, column_name] = y

    return df

In [51]:
data_discover_interpolated = linear_interpolate_gaps(data_discover, column_names_discover, max_gap_length=12)

In [52]:
data_discover_interpolated.iloc[range(1256,1269), [4, 9, 10, 11, 20, 21]]

,bt,bx_gsm,by_gsm,bz_gsm,proton_density,proton_temperature
1256,5.807213,2.268852,-4.409508,2.625738,15.559672,218285.245900
1257,7.372292,-1.107917,-6.313542,2.603542,17.140000,94458.791670
1258,7.163202,-0.125596,-4.368412,2.799675,16.393021,113874.164942
1259,6.954112,0.856724,-2.423283,2.995809,15.646042,133289.538213
1260,6.745022,1.839044,-0.478154,3.191943,14.899062,152704.911485
1261,6.535932,2.821364,1.466976,3.388077,14.152083,172120.284757
1262,6.326842,3.803684,3.412105,3.584211,13.405104,191535.658028
1263,5.303443,3.611967,-2.949180,2.031148,12.658125,210951.031300
1264,5.082787,3.703115,-1.958852,2.768197,13.029016,199954.377000
1265,4.494262,3.891475,-1.589672,1.217869,12.690656,199802.557400


In [53]:
data_discover_interpolated.to_csv("Discover_с_интреполяцией12_актуальные_данные_до_22.09.2024.csv", sep=';', index=False, encoding='utf-8')